In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip uninstall imgaug -y

In [ ]:
!pip install git+https://github.com/aleju/imgaug.git

In [ ]:
%cd /content/drive/MyDrive/DataScience/plate_project_group09

/content/drive/.shortcut-targets-by-id/1Vt0fOz_0sVJoPLSXwaaNYYz64X3oIxL9/DataScience/plate_project_group09


In [ ]:
!cd /content/plate_detector/ && unzip '/content/drive/MyDrive/DataScience/plate_project_group09/datasets/plate1.zip'

In [8]:
import cv2
import numpy as np
from utils import decode_netout
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, Reshape
from losses import YoloLoss
from callbacks import MapEvaluation
from data_generator import BatchGenerator, parse_annotation_xml
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

root = "/content/drive/MyDrive/DataScience/plate_project_group09/"

class PlateDetector:

    def __init__(self):
        self.model = None
        self.anchors = [0.79158,0.99434, 1.22290,1.60963, 1.75330,2.34127, 2.40385,1.27696, 2.60130,3.37916]
        self.num_anchors = len(self.anchors)//2
        self.labels = ['license plates']
        self.num_classes = len(self.labels)
        self.batch_size = 16

    def build_model(self):
        
        backend = InceptionV3(
            include_top = False,
            input_shape = [500, 500, 3],
            weights = root + 'pretrained/inception_backend.h5'
        )
        
        conv_layer_1 = Conv2D(filters = self.num_anchors * (5 + self.num_classes), kernel_size = [1, 1])(backend.output)
        output_layer = Reshape([14, 14, self.num_anchors, 6])(conv_layer_1)


        self.model = Model(backend.input, output_layer)
        self.model.summary()
        
        self.loss = YoloLoss(
            self.anchors,
            [14, 14],
            self.batch_size,
            lambda_obj = 5
        )
        
        self.optimizer = Adam(learning_rate = 1e-4, epsilon=1e-8, decay=0.0)
        run_opts = tf.compat.v1.RunOptions(report_tensor_allocations_upon_oom = True)
        self.model.compile(loss = self.loss, optimizer = self.optimizer)
        
        

    def save_model(self):
        pass  # delete this line and replace yours

    def load_model(self):
        pass  # delete this line and replace yours

    def train(self, **kwargs):
        num_anchors = len(self.anchors)//2
        list_train_images, _ = parse_annotation_xml(
                        '/content/plate_detector/train/anns',
                        '/content/plate_detector/train/images'
        )
        
        list_valid_images, _ = parse_annotation_xml(
                        '/content/plate_detector/valid/anns',
                        '/content/plate_detector/valid/images'
        )
        
        generator_config = {
             'IMAGE_H' : 500,
             'IMAGE_W' : 500,
             'IMAGE_C' : 3,
             'GRID_H'  : 14,
             'GRID_W'  : 14,
             'BOX'     : self.num_anchors,
             'LABELS'  : self.labels,
             'CLASS'   : self.num_classes,
             'ANCHORS' : self.anchors,
             'BATCH_SIZE' : self.batch_size
        }
        
        valid_generator = BatchGenerator(
            list_valid_images, generator_config, preprocess_input = preprocess_input
        )
        train_generator = BatchGenerator(
            list_train_images, generator_config, preprocess_input = preprocess_input
        )
    
        map_evaluation = MapEvaluation(
            self, valid_generator,
            iou_threshold = 0.6,
            save_best = True,
            save_name = root + 'models/plate_best.hdf5'
        )
        
        self.model.fit_generator(
            train_generator,
            steps_per_epoch = int(len(train_generator) * 0.1),
            epochs = 100,
            validation_data = valid_generator,
            validation_steps = int(len(valid_generator) * 0.1),
            callbacks = [map_evaluation]
        )

    def predict(self, image):
        """
        Autotest will call this function
        :param image: a PIL Image object
        :return: a list of boxes, each item is a tuple of (x_min, y_min, x_max, y_max)
        """
        w = image.width
        h = image.height
        image = image.resize([500, 500])
        image_np = np.array(image)
        preprocessed_image = np.array([self.preprocess_input(image_np)])
        netout = self.model.predict(preprocessed_image)[0]
        boxes = decode_netout(
            netout, self.anchors, self.num_classes, 0.5, 0.5
        )
        list_result = []
        for box in boxes:
            xmin = box.xmin * w
            ymin = box.ymin * h
            xmax = box.xmax * w
            ymax = box.ymax * h
            list_result.append((xmin, ymin, xmax, ymax))
        return list_result

    def preprocess_input(self, image):
        return preprocess_input(image)

    def infer(self, image, iou_threshold=0.5, score_threshold=0.5):
        image = cv2.resize(image, (500, 500))
        image = image[..., ::-1]  # make it RGB (it is important for normalization of some backends)

        image = self.preprocess_input(image)
        if len(image.shape) == 3:
            input_image = image[np.newaxis, :]
        else:
            input_image = image[np.newaxis, ..., np.newaxis]

        netout = self.model.predict(input_image)[0]

        boxes = decode_netout(netout, self.anchors, self.num_classes, score_threshold, iou_threshold)

        return boxes


In [9]:
detector = PlateDetector()
detector.build_model()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 500, 500, 3) 0                                            
__________________________________________________________________________________________________
conv2d_95 (Conv2D)              (None, 249, 249, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, 249, 249, 32) 96          conv2d_95[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 249, 249, 32) 0           batch_normalization_94[0][0]     
____________________________________________________________________________________________

In [ ]:
detector.train()

/usr/local/lib/python3.7/dist-packages/imgaug/imgaug.py:188: DeprecationWarning: Function `ContrastNormalization()` is deprecated. Use `imgaug.contrast.LinearContrast` instead.
  warn_deprecated(msg, stacklevel=3)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
8/8 [==============================] - 29s 3s/step - loss: 1461.1873


license plates 0.0000
mAP: 0.0000
mAP did not improve from 0.
Epoch 2/100
8/8 [==============================] - 19s 2s/step - loss: 748.6797


license plates 0.0000
mAP: 0.0000
mAP did not improve from 0.
Epoch 3/100
8/8 [==============================] - 19s 2s/step - loss: 383.8662


license plates 0.0000
mAP: 0.0000
mAP did not improve from 0.
Epoch 4/100
8/8 [==============================] - 20s 3s/step - loss: 214.6025


license plates 0.0000
mAP: 0.0000
mAP did not improve from 0.
Epoch 5/100
8/8 [==============================] - 18s 2s/step - loss: 142.0446


license plates 0.0000
mAP: 0.0000
mAP did not improve from 0.
Epoch 6/100
8/8 [==============================] - 18s 2s/step - loss: 106.3160


license plates 0.0000
mAP: 0.0000
mAP did not improve from 0.
Epoch 7/100
8/8 [==============================] - 18s 2s/step - loss: 101.2938


license plates 0.0000
mAP: 0.0000
mAP did not improve